In [55]:
from icepyx import icesat2data as ipd
import numpy as np
import os
import shutil
import h5py
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import sys
import pyproj
import pointCollection as pc

%matplotlib widget

In [27]:
short_name = 'ATL06'
spatial_extent = [31.5, -70.56, 33.73, -69.29]
date_range = ['2020-03-30','2020-04-1']
region_a = ipd.Icesat2Data(short_name, spatial_extent, date_range)
region_a.earthdata_login('kmsejan','kmsejan@uu.nl')

Earthdata Login password:  ·············


In [28]:
#region_a.order_vars.avail()

In [29]:
region_a.order_vars.append(var_list=['count'])

In [30]:
region_a.download_granules('/home/jovyan/surface_classification/data') 

Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSIDC
order ID:  5000000701676
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
Beginning download of zipped output...
Data request 5000000701676 of  1  order(s) is downloaded.
Download complete


In [68]:

FILE_NAME = '/home/jovyan/data/processed_ATL06_20200330121520_00600712_003_01.h5'
f = h5py.File(FILE_NAME, mode='r') 

count = f['gt1l/residual_histogram/count'][:] # has units of n_histograms, n_bins
lat_mean = f['gt1l/residual_histogram/lat_mean'][:]
lon_mean = f['gt1l/residual_histogram/lon_mean'][:]
h_li = f['gt1l/land_ice_segments/h_li'][:]
h_lat = f['gt1l/land_ice_segments/latitude'][:]
h_lon = f['gt1l/land_ice_segments/longitude'][:]

#latitude = f['/gt2r/heights/lat_ph']
#longitude = f['/gt2r/heights/lon_ph']
#height = f['gt2r/heights/h_ph']

Removing the data far from surface in each histogram.

In [46]:
data = count[:, 200:550]

In [47]:

fig=plt.figure(figsize=(10,8))
ax = fig.add_subplot(111)
h = ax.imshow(data,vmin=0,vmax=30,cmap='Reds')
plt.colorbar(h)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<h2>Plot research area of the above file</h2>

** still needs track on this image

In [56]:
data_root='/srv/tutorial-data/land_ice_applications/'

In [57]:
! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
sys.path.append(os.path.join(os.getcwd(), '..'))

In [58]:
spatial_extent = np.array(spatial_extent)
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
MOA=pc.grid.data().from_geotif(os.path.join(data_root, 'MOA','moa_2009_1km.tif'), bounds=[XR, YR])

# show the mosaic:
plt.figure()
MOA.show(cmap='gray', clim=[14000, 17000])
plt.plot(xy[0,:], xy[1,:])
plt.title(' Area used for search')

[-70.56 -69.29 -69.29 -70.56 -70.56]
[33.73 33.73 31.5  31.5  33.73]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'gray', 'clim': [14000, 17000], 'extent': array([1114050., 1262050., 1773825., 1938825.]), 'origin': 'lower'}


Text(0.5, 1.0, ' Area used for search')

<h2> A different way of plotting track on map</a>

In [69]:
spatial_extent_ps = [spatial_extent[0], spatial_extent[2], spatial_extent[1], spatial_extent[3]]

## we will want to set colorbar parameters based on the chosen variable
vmin=0
vmax=6
ticks=np.arange(vmin,vmax+1,1)

plt.figure(figsize=(8,8), dpi= 90)
ax = plt.axes(projection=ccrs.SouthPolarStereo(central_longitude=0)) # choose polar sterographic for projection
ax.coastlines(resolution='50m', color='black', linewidth=1)
ax.set_extent(spatial_extent_ps, ccrs.PlateCarree())
plt.plot(lon_mean,lat_mean,transform=ccrs.PlateCarree())
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot comparing mean_lon and mean_lon from histrograms with beam lat and lon

In [70]:
plt.figure()
plt.plot(h_lon,h_lat,'ob' )
plt.plot(lon_mean, lat_mean,'.r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …